O objetivo é realizar uma análise espacial do perfil predominante das empresas de Data Science nos estados brasileiros presentes no dataset de acordo com parâmetros de **caracterização**.

A caracterização das empresas vai ser feita a partir dos campos:
* quantidade de funcionários
* cargo
* salário
* idade
* gênero

Para cada campo, foi feita a análise entre Geral (qualquer 'is_data_science_professional') e Profissional de DS ('is_data_science_professional' = 1).

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import re
%matplotlib inline

pd.set_option('display.max_columns', 200)

In [ ]:
respostas = pd.read_csv('../input/pesquisa-data-hackers-2019/datahackers-survey-2019-anonymous-responses.csv')
respostas.head()

Vamos analisar quantos registros do dataset são de funcionários trabalhando no Brasil.

In [ ]:
mask_br = respostas["('P3', 'living_in_brasil')"] == 1
respostas_br = respostas[mask_br]
respostas_fora = respostas[~mask_br]
respostas_br.shape, respostas_fora.shape

Temos a grande maioria de registros de pessoas que moram no Brasil. Vamos utilizar esses registros para a caracterização.

## Caracterização

Capturamos os campos relacionados a essa análise.

In [ ]:
campos_caract = ["('P5', 'living_state')", "('P19', 'is_data_science_professional')", "('P12', 'workers_number')", "('D6', 'anonymized_role')", "('P16', 'salary_range')", "('P1', 'age')", "('P2', 'gender')"]
respostas_br_caract = respostas_br[campos_caract].dropna(subset=["('P5', 'living_state')"])
respostas_br_caract["('P5', 'living_state')"].value_counts()

Vamos tomar também o subset com registros de quem se considera profissional de DS no momento da pesquisa.

In [ ]:
respostas_br_ds_caract = respostas_br_caract[respostas_br_caract["('P19', 'is_data_science_professional')"] == True]
respostas_br_ds_caract["('P5', 'living_state')"].value_counts()

Infelizmente não temos todos os estados contemplados no resultado final da pesquisa, e temos alguns estados com poucos registros, como ES, RS e SC. Para esses estados será mais difícil tirar conclusões com base nos dados.

Analisaremos para todos os estados na lista.

### Quantidade de funcionários

* Geral

In [ ]:
respostas_br_caract["('P12', 'workers_number')"].value_counts()

In [ ]:
col_ordem = ['Minas Gerais (MG)', 'São Paulo (SP)', 'Paraná (PR)', 'Rio Grande do Sul (RS)', 'Rio de Janeiro (RJ)', 'Espírito Santo (ES)', 'Santa Catarina (SC)']
ordem = ['de 1 a 5', 'de 6 a 10', 'de 11 a 50', 'de 51 a 100', 'de 101 a 500', 'de 501 a 1000', 'de 1001 a 3000', 'Acima de 3000']
ax = sns.catplot(x="('P12', 'workers_number')", data=respostas_br_caract, col="('P5', 'living_state')", col_wrap=4, kind="count", sharex=False, order=ordem, col_order=col_ordem)
ax.set_xticklabels(rotation=65, horizontalalignment='right')
ax.set(ylabel='Total de funcionários')
plt.tight_layout()
plt.subplots_adjust(top=0.9)
ax.fig.suptitle('Quantidade de funcionários nas empresas - Geral', fontsize='18', fontweight='bold')

A partir dos gráficos:

* SP, RJ e RS apresentam a maioria de suas empresas com mais de 3000 de funcionários.
* PR, MG e SC apresentam a maioria de suas empresas entre 101 e 500 funcionarios.

* Profissionais de DS

In [ ]:
respostas_br_ds_caract["('P12', 'workers_number')"].value_counts()

In [ ]:
ordem = ['de 1 a 5', 'de 6 a 10', 'de 11 a 50', 'de 51 a 100', 'de 101 a 500', 'de 501 a 1000', 'de 1001 a 3000', 'Acima de 3000']
ax = sns.catplot(x="('P12', 'workers_number')", data=respostas_br_ds_caract, col="('P5', 'living_state')", col_wrap=4, kind="count", sharex=False, order=ordem, col_order=col_ordem)
ax.set_xticklabels(rotation=65, horizontalalignment='right')
ax.set(ylabel='Total de funcionários')
plt.tight_layout()
plt.subplots_adjust(top=0.9)
ax.fig.suptitle('Quantidade de funcionários nas empresas - Profissionais de DS', fontsize='18', fontweight='bold')

A maior parte dos funcionários que responderam à pesquisa estão em empresas de Grande Porte, o que sugere que elas podem estar investindo mais na contratação desses profissionais, por terem um time ou uma infraestrutura preparada para projetos de DS, por exemplo.

Percebemos que SP e MG se destacam no contexto de Data Science, apresentando o maior contingente de empresas de Grande Porte no ramo.

(Grande Porte: Mais de 100 funcionários de acordo com a Sebrae: https://blog.sebrae-sc.com.br/numero-de-empregados-receita-bruta-para-mei-me-epp/)

## Cargo

* Geral

In [ ]:
respostas_br_caract["('D6', 'anonymized_role')"].value_counts()

In [ ]:
labels=['Outras', 'Analista de Dados', 'Analista de BI', 'Cientista de Dados', 'Analista de IM', 'Engenheiro', 'Dev. ou Eng. de Software',
       'Business Analyst', 'Engenheiro de Dados', 'Estatístico', 'Analista de Marketing', 'DBA/Adm. de BD', 'Eng. de ML', 'Economista']
ax = sns.catplot(x="('D6', 'anonymized_role')", data=respostas_br_caract, col="('P5', 'living_state')", col_wrap=4, kind="count", sharex=False, col_order=col_ordem)
ax.set_xticklabels(labels=labels, rotation=65, horizontalalignment='right')
ax.set(ylabel='Total de funcionários')
plt.tight_layout()
plt.subplots_adjust(top=0.9)
ax.fig.suptitle('Cargos - Geral', fontsize='18', fontweight='bold')

A partir dos gráficos:

* Boa parte dos estados tem um contingente de "Dev. ou Eng. de Sofware" e "Outras" grande, como área de atuação no momento da pesquisa.
* SP e RJ apresentaram quantidades de "Cientista de Dados" e "Analista de Dados" que superam a categoria de "Dev. e Eng. de Sofware", apresentando ao mesmo tempo uma quantidade de "Engenheiros de Dados" relativamente importante.
* MG, PR e SC apresentaram o maior contingente de "Dev. e Eng. de Sofware" dentre os outros cargos.

Isso pode significar que pode estar acontecendo uma migração de carreira da área de "Dev. e Eng. de Sofware" para as áreas ligadas à Ciência de Dados. O fato de vários Desenvolvedores terem preenchido o formulário do DataHackers sugere um interesse desses profissionais no domínio de Ciência de Dados, o que pode significar uma possível mudança de carreira.

Os dados sugerem que em SP e no RJ essa migração é responsável por um contingente maior nas áreas ligadas à DS quando comparadas à Desenvolvimento de Software, e que em MG, PR, SC essa transformação está acontecendo.

* Profissionais de DS

In [ ]:
respostas_br_ds_caract["('D6', 'anonymized_role')"].value_counts()

In [ ]:
labels=['Outras', 'Analista de Dados', 'Analista de BI', 'Cientista de Dados', 'Analista de IM', 'Engenheiro', 'Dev. ou Eng. de Software',
       'Business Analyst', 'Engenheiro de Dados', 'Estatístico', 'Analista de Marketing', 'DBA/Adm. de BD', 'Eng. de ML', 'Economista']
ax = sns.catplot(x="('D6', 'anonymized_role')", data=respostas_br_ds_caract, col="('P5', 'living_state')", col_wrap=4, kind="count", sharex=False, col_order=col_ordem)
ax.set_xticklabels(labels=labels, rotation=65, horizontalalignment='right')
ax.set(ylabel='Total de funcionários')
plt.tight_layout()
plt.subplots_adjust(top=0.9)
ax.fig.suptitle('Cargos - Profissionais de DS', fontsize='18', fontweight='bold')

Dos profissionais que se consideram Profissionais de DS, a maioria trabalha atualmente **como Data Scientist, como Desenv. ou Eng. Software , ou em outros cargos.**

Interessante que, neste campo, quase não constam Data Engineers, provavelmente porque a pergunta no formulário sobre "Se você se considera um profissional de DS?" foi interpretada de modo específico como "Data Scientist", e não como na área de Data Science de modo geral, o que contemplaria todos os domínios correlatos da área.

## Salário

* Geral

In [ ]:
respostas_br_caract["('P16', 'salary_range')"].value_counts()

In [ ]:
def get_media_salarios_aparada(salary_range_str):
    search = re.search(r"de R\$ (\d+\.?\d+?)\/mês a R\$ (\d+\.?\d+?)\/mês", str(salary_range_str))
    salary_range = []
    if (search is not None):
        salary_range = list(search.groups())
    else:
        return 0
    
    if "." in salary_range[0]:
        salary_range[0] = salary_range[0].replace(".", "")
    
    if "." in salary_range[1]:
        salary_range[1] = salary_range[1].replace(".", "")
    
    return round((int(salary_range[0]) + int(salary_range[1])) / 2)

In [ ]:
ordem=['Menos de R$ 1.000/mês', 'de R$ 1.001/mês a R$ 2.000/mês', 'de R$ 2.001/mês a R$ 3000/mês',  'de R$ 3.001/mês a R$ 4.000/mês',
       'de R$ 4.001/mês a R$ 6.000/mês', 'de R$ 6.001/mês a R$ 8.000/mês', 'de R$ 8.001/mês a R$ 12.000/mês', 'de R$ 12.001/mês a R$ 16.000/mês',
       'de R$ 16.001/mês a R$ 20.000/mês', 'de R$ 20.001/mês a R$ 25.000/mês', 'Acima de R$ 25.001/mês']
labels=['< R$ 1.000', 'R$ 1.001 a 2.000', 'R$ 2.001 a 3.000', 'R$ 3.001 a 4.000', 'R$ 4.001 a 6.000', 'R$ 6.001 a 8.000', 'R$ 8.001 a 12.000',
         'R$ 12.001 a 16.000', 'R$ 16.001 a 20.000', 'R$ 20.001 a 25.000', '> R$ 25.001']
ax = sns.catplot(x="('P16', 'salary_range')", data=respostas_br_caract, col="('P5', 'living_state')", col_wrap=4, kind="count", sharex=False, order=ordem, col_order=col_ordem)
ax.set_xticklabels(labels=labels, rotation=65, horizontalalignment='right')
ax.set(ylabel='Total de funcionários')
plt.tight_layout()
plt.subplots_adjust(top=0.9)
ax.fig.suptitle('Salários (por mês) - Geral', fontsize='18', fontweight='bold')

In [ ]:
df_aux = respostas_br_caract.copy()
df_aux["('P16', 'salary_range')"] = df_aux["('P16', 'salary_range')"].apply(get_media_salarios_aparada)
df_aux = pd.DataFrame(df_aux.groupby("('P5', 'living_state')")["('P16', 'salary_range')"].mean().sort_values(ascending=False))
df_aux = df_aux.rename(columns={"('P16', 'salary_range')": "Média Geral"})
df_aux

Os salários mais frequentes se concentram entre R\\$ 3.000 e R\\$ 12.000/mês, sendo que

* MG, RJ, PR e SC têm a maioria dos funcionários ganhando na faixa de R\\$ 4.001/mês a R\\$ 6.000/mês.
* SP apresenta um contingente bastante importante na faixa de R\\$ 8.001 e R\\$ 12.000/mês.

Aqui é interessante verificar a correlação com o **contingente de cargos**, para saber quais cargos têm os maiores salários em cada estado.

O mesmo pode ser feito em relação à qtde de funcionarios, para ter uma ideia da correlação entre o **porte da empresa** e os salários. (Análise de Correlação).

SP e RJ apresentam os maiores salários médios.

(O cálculo foi feito tomando a média dos extremos de cada classe como valor médio da classe, e ponderando para cada estado pelo "counts" de cada classe).

Um ponto interessante seria verificar a **correlação entre salário e sexo, salário e idade.**

* Profissionais de DS

In [ ]:
respostas_br_ds_caract["('P16', 'salary_range')"].value_counts()

In [ ]:
ordem=['Menos de R$ 1.000/mês', 'de R$ 1.001/mês a R$ 2.000/mês', 'de R$ 2.001/mês a R$ 3000/mês',  'de R$ 3.001/mês a R$ 4.000/mês',
       'de R$ 4.001/mês a R$ 6.000/mês', 'de R$ 6.001/mês a R$ 8.000/mês', 'de R$ 8.001/mês a R$ 12.000/mês', 'de R$ 12.001/mês a R$ 16.000/mês',
       'de R$ 16.001/mês a R$ 20.000/mês', 'de R$ 20.001/mês a R$ 25.000/mês', 'Acima de R$ 25.001/mês']
labels=['< R$ 1.000', 'R$ 1.001 a 2.000', 'R$ 2.001 a 3.000', 'R$ 3.001 a 4.000', 'R$ 4.001 a 6.000', 'R$ 6.001 a 8.000', 'R$ 8.001 a 12.000',
         'R$ 12.001 a 16.000', 'R$ 16.001 a 20.000', 'R$ 20.001 a 25.000', '> R$ 25.001']
ax = sns.catplot(x="('P16', 'salary_range')", data=respostas_br_ds_caract, col="('P5', 'living_state')", col_wrap=4, kind="count", sharex=False, order=ordem, col_order=col_ordem)
ax.set_xticklabels(labels=labels, rotation=65, horizontalalignment='right')
ax.set(ylabel='Total de funcionários')
plt.tight_layout()
plt.subplots_adjust(top=0.9)
ax.fig.suptitle('Salários (por mês) - Profissionais de DS', fontsize='18', fontweight='bold')

In [ ]:
df_aux_ds = respostas_br_ds_caract.copy()
df_aux_ds["('P16', 'salary_range')"] = df_aux_ds["('P16', 'salary_range')"].apply(get_media_salarios_aparada)
df_aux_ds = pd.DataFrame(df_aux_ds.groupby("('P5', 'living_state')")["('P16', 'salary_range')"].mean().sort_values(ascending=False))
df_aux_ds = df_aux_ds.rename(columns={"('P16', 'salary_range')": "Média DS"})
df_aux = df_aux.merge(df_aux_ds, left_on="('P5', 'living_state')", right_on="('P5', 'living_state')")
df_aux["Estado"] = df_aux.index

In [ ]:
df_aux_melted = df_aux.melt('Estado', var_name='cols',  value_name='vals')
df_aux_melted

In [ ]:
ax = sns.catplot(x="Estado", y="vals", data=df_aux_melted, hue='cols', kind="bar", order=col_ordem)
ax.set_xticklabels(rotation=65, horizontalalignment='right')
ax.set(ylabel='Salário médio (em R$)')
plt.subplots_adjust(top=0.9)
ax.fig.suptitle('Salários médios (por mês)', fontsize='18', fontweight='bold')

A concentração de salários, para profissionais de DS, se mantém no mesmo intervalo de R\\$ 3.001 a 12.000, porém notamos que:

* SP apresenta mais funcionários com salários na faixa de R\\$ 8.001 a 12.000
* MG, PR e RJ apresentam mais funcionários com salários na faixa de R\\$ 4.001 a 6.000
* SC apresenta mais funcionários com salários nas faixas de R\\$ 4.001 a 6.000 e R\\$ 6.001 a 8.000

Os salários médios dos profissionais de DS são maiores em todos os estados da pesquisa, exceto em SC.

## Idade e Sexo

* Global

Contemplando todos os estados:

In [ ]:
ax = sns.distplot(respostas_br_caract["('P1', 'age')"])
ax.xaxis.set_major_locator(ticker.MultipleLocator(2))

In [ ]:
print("Média de Idade Global: {}".format(respostas_br_caract["('P1', 'age')"].mean()))
print("Média de Idade Masculino: {}".format(respostas_br_caract[respostas_br_caract["('P2', 'gender')"] == "Masculino"]["('P1', 'age')"].mean()))
print("Média de Idade Feminino: {}".format(respostas_br_caract[respostas_br_caract["('P2', 'gender')"] == "Feminino"]["('P1', 'age')"].mean()))

Média de idade global, do público Masculino e do Feminino são todas de aproximadamente **29 anos.**

In [ ]:
sns.catplot(x="('P2', 'gender')", data=respostas_br_caract, kind="count")

Essa primeira análise sugere que o público da pesquisa é majoritariamente masculino (=~ 5 vezes maior), entre 23 e 32 anos.

Para medir a participação feminina, vamos calcular as proporções de mulheres em cada estado:

In [ ]:
mask_homem = respostas_br_caract["('P2', 'gender')"] == 'Masculino'
mask_mulher = respostas_br_caract["('P2', 'gender')"] == 'Feminino'
(respostas_br_caract[mask_mulher].groupby(["('P5', 'living_state')"])["('P2', 'gender')"].value_counts() / respostas_br_caract.groupby(["('P5', 'living_state')"])["('P2', 'gender')"].count()).sort_values(ascending=False)

* ES é o que tem a maior participação relativa de mulheres em DS (apesar de ter poucos registros...), seguido do RJ e MG.

In [ ]:
mask = respostas_br_caract["('P2', 'gender')"] == 'Masculino'
ax = sns.catplot(x="('P1', 'age')", data=respostas_br_caract[mask], col="('P5', 'living_state')", col_wrap=4, kind="count", sharex=False)
ax.set_xticklabels(rotation=90, horizontalalignment='right')
ax.set(ylabel='Total de funcionários')
plt.tight_layout()
plt.subplots_adjust(top=0.9)
ax.fig.suptitle('Idade dos funcionários - Masculino', fontsize='18', fontweight='bold')

In [ ]:
respostas_br_caract[respostas_br_caract["('P2', 'gender')"] == 'Masculino'].groupby(["('P5', 'living_state')"])["('P1', 'age')"].mean().sort_values(ascending=False)

* Em MG, SP e PR, existe um contingente significativo de DSs do sexo Masculino que tem idade < 20 ou > 45.


* Analisando somente as médias do DSs do sexo Masculino: RJ, PR e SP superam (~ 30 anos) a média de 29 anos (ES tem aproximadamente 29 anos) => não dá pra dizer que essa diferença é significativa.


In [ ]:
mask = respostas_br_caract["('P2', 'gender')"] == 'Feminino'
ax = sns.catplot(x="('P1', 'age')", data=respostas_br_caract[mask], col="('P5', 'living_state')", col_wrap=4, kind="count", sharex=False, col_order=col_ordem)
ax.set_xticklabels(rotation=90, horizontalalignment='right')
ax.set(ylabel='Total de funcionários')
plt.tight_layout()
plt.subplots_adjust(top=0.9)
ax.fig.suptitle('Idade dos funcionários - Feminino', fontsize='18', fontweight='bold')

Média de idade do público Feminino, por estado:

In [ ]:
respostas_br_caract[respostas_br_caract["('P2', 'gender')"] == 'Feminino'].groupby(["('P5', 'living_state')"])["('P1', 'age')"].mean().sort_values(ascending=False)

Temos uma diferença significativa de idade do público Feminino em relação à média de idade (29 anos) no PR e em SC.

* Profissionais de DS

Contemplando todos os estados:

In [ ]:
ax = sns.distplot(respostas_br_ds_caract["('P1', 'age')"])
ax.xaxis.set_major_locator(ticker.MultipleLocator(2))

O pico em 22 e 23 anos é mais significativo em relação à análise global.

Pode mostrar um interesse maior em DS pelo profissionais que estão entrando no mercado de trabalho.

In [ ]:
print("Média de Idade Global: {}".format(respostas_br_ds_caract["('P1', 'age')"].mean()))
print("Média de Idade Masculino: {}".format(respostas_br_ds_caract[respostas_br_ds_caract["('P2', 'gender')"] == "Masculino"]["('P1', 'age')"].mean()))
print("Média de Idade Feminino: {}".format(respostas_br_ds_caract[respostas_br_ds_caract["('P2', 'gender')"] == "Feminino"]["('P1', 'age')"].mean()))

Média de idade global, do público Masculino e do Feminino que se consideram profissionais de DS são todas de aproximadamente **29 anos**, como no caso global.

In [ ]:
sns.catplot(x="('P2', 'gender')", data=respostas_br_ds_caract, kind="count", order=['Masculino', 'Feminino'])

In [ ]:
razao_global = len(respostas_br_caract[respostas_br_caract["('P2', 'gender')"] == 'Masculino']) / len(respostas_br_caract[respostas_br_caract["('P2', 'gender')"] == 'Feminino'])
razao_ds = len(respostas_br_ds_caract[respostas_br_caract["('P2', 'gender')"] == 'Masculino']) / len(respostas_br_ds_caract[respostas_br_caract["('P2', 'gender')"] == 'Feminino'])
print("Proporção global entre homens e mulheres: {}".format(razao_global))
print("Proporção em DS entre homens e mulheres: {}".format(razao_ds))                                                                                               

Notamos uma proporção parecida entre homens e mulheres no caso geral e dos que se consideram profissionais de DS.

Medindo a participação Feminina:

In [ ]:
mask_homem = respostas_br_ds_caract["('P2', 'gender')"] == 'Masculino'
mask_mulher = respostas_br_ds_caract["('P2', 'gender')"] == 'Feminino'
(respostas_br_ds_caract[mask_mulher].groupby(["('P5', 'living_state')"])["('P2', 'gender')"].value_counts() / respostas_br_ds_caract.groupby(["('P5', 'living_state')"])["('P2', 'gender')"].count()).sort_values(ascending=False)

* RJ, MG e ES também apresentam maior participação relativa feminina em DS.

In [ ]:
mask = respostas_br_ds_caract["('P2', 'gender')"] == 'Masculino'
ax = sns.catplot(x="('P1', 'age')", data=respostas_br_ds_caract[mask], col="('P5', 'living_state')", col_wrap=4, kind="count", sharex=False, col_order=col_ordem)
ax.set_xticklabels(rotation=90, horizontalalignment='right')
ax.set(ylabel='Total de funcionários')
plt.tight_layout()
plt.subplots_adjust(top=0.9)
ax.fig.suptitle('Idade dos funcionários - Masculino', fontsize='18', fontweight='bold')

* MG e SP apresentam maior contingente relativo entre 22 e 25 anos, do sexo Masculino.
* PR, RS e RJ têm maior contingente relativo entre 28 e 30 anos, do sexo Masculino.

In [ ]:
mask = respostas_br_ds_caract["('P2', 'gender')"] == 'Feminino'
ax = sns.catplot(x="('P1', 'age')", data=respostas_br_ds_caract[mask], col="('P5', 'living_state')", col_wrap=4, kind="count", sharex=False, col_order=col_ordem)
ax.set_xticklabels(rotation=90, horizontalalignment='right')
ax.set(ylabel='Total de funcionários')
plt.tight_layout()
plt.subplots_adjust(top=0.9)
ax.fig.suptitle('Idade dos funcionários - Feminino', fontsize='18', fontweight='bold')

* SP apresenta mais mulheres entre 24 e 25 anos e 29 a 31 anos.
* MG apresenta mais mulheres entre 30 e 32 anos.
* Alguns estados mostraram muitos participantes com mais de 40 anos, como MG, SP, PR (Fem. e Masc.), RJ (Fem.) e RS (Masc.).

Média de idade do público Feminino que se consideram profissionais de DS, por estado:

In [ ]:
respostas_br_ds_caract[respostas_br_caract["('P2', 'gender')"] == 'Feminino'].groupby(["('P5', 'living_state')"])["('P1', 'age')"].mean().sort_values(ascending=False)

Comparação das médias de idades entre Geral e DS:

In [ ]:
for sexo in ['Masculino', 'Feminino']:
    df_aux = respostas_br_caract.copy()
    df_aux = pd.DataFrame(respostas_br_caract[respostas_br_caract["('P2', 'gender')"] == sexo].groupby(["('P5', 'living_state')"])["('P1', 'age')"].mean().sort_values(ascending=False))
    df_aux = df_aux.rename(columns={"('P1', 'age')": "Média Geral"})

    df_aux_ds = respostas_br_ds_caract.copy()
    df_aux_ds = pd.DataFrame(respostas_br_ds_caract[respostas_br_ds_caract["('P2', 'gender')"] == sexo].groupby(["('P5', 'living_state')"])["('P1', 'age')"].mean().sort_values(ascending=False))
    df_aux_ds = df_aux_ds.rename(columns={"('P1', 'age')": "Média DS"})
    df_aux = df_aux.merge(df_aux_ds, left_on="('P5', 'living_state')", right_on="('P5', 'living_state')")
    df_aux["Estado"] = df_aux.index

    df_aux_melted = df_aux.melt('Estado', var_name='cols',  value_name='vals')

    ax = sns.catplot(x="Estado", y="vals", data=df_aux_melted, hue='cols', kind="bar", order=col_ordem)
    ax.set_xticklabels(rotation=65, horizontalalignment='right')
    ax.set(ylabel='Média das idades')
    plt.subplots_adjust(top=0.9)
    ax.fig.suptitle('Média das idades - {}'.format(sexo), fontsize='18', fontweight='bold')

As médias do público Feminino apresentam variância maior ao redor da média global de DS.

### Análise de correlação

Foi utilizada a correlação de Pearson e fazer um mapa de calor para as variáveis de caracterização, para os registros de quem se considera profissionais da área de DS.

In [ ]:
from category_encoders import OneHotEncoder

In [ ]:
def get_media_salarios(salary_range_str):
    search = re.search(r"de R\$ (\d+\.?\d+?)\/mês a R\$ (\d+\.?\d+?)\/mês", str(salary_range_str))
    search_acima = re.search(r"Acima de R\$ (\d+\.?\d+?)\/mês", str(salary_range_str))
    search_abaixo = re.search(r"Menos de R\$ (\d+\.?\d+?)\/mês", str(salary_range_str))
    salary_range = []
    if (search is not None):
        salary_range = list(search.groups())
    elif (search_acima is not None):
        return 30000
    elif (search_abaixo is not None):
        return 500
    else:
        return 0
    
    if "." in salary_range[0]:
        salary_range[0] = salary_range[0].replace(".", "")
    
    if "." in salary_range[1]:
        salary_range[1] = salary_range[1].replace(".", "")
    
    return round((int(salary_range[0]) + int(salary_range[1])) / 2)

def get_media_funcionarios(func_range_str):
    search = re.search(r"de (\d+) a (\d+)", str(func_range_str))
    search_acima = re.search(r"Acima de (\d+)", str(func_range_str))
    func_range = []
    if (search is not None):
        func_range = list(search.groups())
    elif (search_acima is not None):
        return 3250
    else:
        return 0
    
    return round((int(func_range[0]) + int(func_range[1])) / 2)

def transform_gender(gender):
    return int(gender == 'Feminino')

In [ ]:
df_aux = respostas_br_ds_caract.copy()
df_aux["('P16', 'salary_range')"] = df_aux["('P16', 'salary_range')"].apply(get_media_salarios)
df_aux["('P2', 'gender')"] = df_aux["('P2', 'gender')"].apply(transform_gender)
df_aux["('P12', 'workers_number')"] = df_aux["('P12', 'workers_number')"].apply(get_media_funcionarios)
ohe_cargo = OneHotEncoder(cols=["('D6', 'anonymized_role')"], use_cat_names=True, drop_invariant=True)
df_aux = ohe_cargo.fit_transform(df_aux)
df_aux = df_aux.drop("('D6', 'anonymized_role')_nan", axis=1)
df_aux.head()

Análise com todos os estados:

In [ ]:
corr = df_aux.corr()
sns.heatmap(corr)

A partir do mapa de calor, notamos que:
* "Salário" e "Idade" têm correlação positiva da ordem de 0.4. Isso tem coerência se pensarmos que quanto maior a idade, em geral, maior a experiência do profissional, o que pode levar a um cargo com salário mais elevado.
* "Salário" e "Qtde de funcionários" também têm correlação positiva da ordem de 0.4. Isso sugere que empresas com mais funcionários costumam remunerar melhor os funcionarios de Data Science.

## Conclusão

Reunindo os principais pontos, por estado:

* MG

  - Maioria das empresas em que estão os participantes tem entre 101 e 500 funcionários, seguido de empresas com mais de 3.000 funcionários.
      - Grande contingente de participantes em empresas de Grande Porte no ramo de DS.
  - Maior contingente de "Dev. e Eng. de Sofware" dentre os outros cargos (possível migração de carreira).
  - Maioria dos salários na faixa de R\\$ 4.001 a R\\$ 6.000/mês.
  - Maior contingente entre 22 e 25 anos, do sexo Masculino (Profissionais de DS).
  - Maior contingente entre 30 e 32 anos, do sexo Feminino (Profissionais de DS).
  

* SP

    - Maioria das empresas em que estão os participantes tem mais de 3000 de funcionários.
        - Grande contingente de participantes em empresas de Grande Porte no ramo de DS.
    - Quantidade de "Cientistas de Dados" e "Analistas de Dados" que superam a categoria de "Dev. e Eng. de Sofware", apresentando ao mesmo tempo uma quantidade de "Engenheiros de Dados" relativamente importante.
    - Maioria dos salários na faixa de R\\$ 8.001 e R\\$ 12.000/mês.
    - Maior salário médio (seja no caso geral, seja nos casos dos que se declararam Profissionais de DS).
    - Maior contingente relativo entre 22 e 25 anos, do sexo Masculino (Profissionais de DS).
    - Maior contingente relativo entre 24 e 25 anos e 29 a 31 anos, do sexo Feminino (Profissionais de DS).

* PR

    - Maioria das empresas em que estão os participantes tem entre 101 e 500 funcionários.
    - Maior contingente de "Dev. e Eng. de Sofware" dentre os outros cargos (possível migração de carreira).
    - Maioria dos salários na faixa de R\\$ 4.001 a 6.000/mês.
    - Maior contingente entre 28 e 30 anos, seja do sexo Masculino (Profissionais de DS).
    - Maiores médias de idade de participantes do sexo Feminino (~ 31 anos).

* RS

    - Maioria das empresas em que estão os participantes tem entre 101 e 500 ou 3.000 funcionários.
    - Maior contingente de "Analista de BI" e "Cientista de Dados", contingente pequeno de "Eng. de Dados".
    - Distribuição uniforme de salários na faixa de R\\$ 3.000 a 12.000/mês.
    - Maior contingente entre 28 e 30 anos, do sexo Masculino (Profissionais de DS).
    - Poucos registros (4) de participantes do sexo Feminino (Profissionais de DS).

* RJ

    - Maioria das empresas em que estão os participantes tem mais de 3000 de funcionários.
        - Grande contingente de participantes em empresas de Grande Porte no ramo de DS.
    - Quantidade de "Cientistas de Dados" e "Analistas de Dados" que superam a categoria de "Dev. e Eng. de Sofware", apresentando ao mesmo tempo uma quantidade de "Engenheiros de Dados" relativamente importante.
    - Maioria dos salários na faixa de R\\$ 4.001/mês a R\\$ 6.000/mês.
    - Salário médio (seja no caso geral, seja nos casos dos que se declararam Profissionais de DS) de R\\$ 5.000.
    - Maior contingente entre 28 e 30 anos, do sexo Masculino e Feminino (Profissionais de DS).

* ES

    Devido à pouca quantidade de dados do ES (27 registros), ficou difícil determinar o perfil para as variáveis estabelecidas.

* SC

    * Maioria das empresas em que estão os participantes tem entre 101 e 3000 de funcionários.
    * Maior contingente de "Dev. e Eng. de Sofware" dentre os outros cargos (possível migração de carreira).
    * Maioria dos salários na faixa de R\\$ 4.001 a 8.000.
    * Distribuição aproximadamente uniforme entre 23 e 35 anos (Profissionais de DS).
    * Poucos registros (6) de participantes do sexo Feminino (Profissionais de DS).